In [17]:
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

In [18]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom = NULL, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: PLS-LDA (plsgenomics)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [19]:
add_model_to_manifest <- function(
    model_id, 
    outlier = "", 
    preprocessing="",
    source="", 
    weight = "",
    n = "",
    oob_error = "",
    accuracy = "",
    r2 = "",
    chi2prob = "",
    logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- "source,outliers,preprocessing,weight,n,oob,accuracy,r2,chi2prob,model_id"
        write(header, file = logpath)
    }

    line <- paste(
        source,
        outlier,
        preprocessing,
        weight,
        n,
        oob_error,
        accuracy,
        r2,
        chi2prob,
        sep=","
    )
    line <- paste0(line, ",", model_id)

    write(line, file=logpath, append = TRUE)
}

In [20]:
train_pls_lda <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    ncomp = 32,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    save_path = "./mle/models",
    seed = NULL,
    log_string = ""
){

    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        x_test <- preprocess_fn(x_test)
    }

    sample_perm <- permute::shuffle(length(train_labels))
    sample_index <- create_stratified_sample(
        train_labels,
        permutation = sample_perm,
        samples_per_pft = 47
    )
    x_train <- x_train[sample_perm,][sample_index,]
    train_labels <- train_labels[sample_perm][sample_index]

    #if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
    levels(test_labels) <- c(levels(test_labels), "Forb")
    #    }
    
    model_s3_obj <- list(
        x_train = x_train,
        y_train = train_labels,
        ncomp = ncomp,
        nruncv = 0
    )

    class(model_s3_obj) <- "pls_lda"

    save(
        model_s3_obj,
        file = file.path(
            save_path, 
            paste0(model_id, ".rda"))
    )

    # create predictions (ranger)
    model_predictions <- predict(
            model_s3_obj, 
            x_test
        )

    print(model_predictions)


    # generate the confusion matrix

    confusion_matrix <- caret::confusionMatrix(
        model_predictions %>% as.factor(),
        test_labels,
        mode = "everything"
    )

    log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = log_string,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_rf_n.log"
            )

    return(
        list(
                model = model_s3_obj,
                confusion = confusion_matrix %>% as.list()
                )
    )

    

}

In [21]:
apply_model.pls_lda <- function(x, model,  ...){
    
    if("x" %in% colnames(x) && "y" %in% colnames(x)){
        print("Spatial ifnormation detected, ignoring....")
        target_df <- subset(x, select=-c(x,y))
        prediction_df <- predict.pls_lda(model, target_df) %>% as.data.frame()
        prediction_df$x <- x$x
        prediction_df$y <- x$y

    } else {
        prediction_df <- predict.pls_lda(model, x) %>% as.data.frame()
    }

    return(prediction_df)
}

In [22]:
write_pls_lda_model <- function(object, save_path, uuid = NULL){
    model_id <- uuid
    if(is.null(model_id)){
        model_id <- uuid::UUIDgenerate()
    }

    save(x_train, x_train_path)
}

In [23]:
predict.pls_lda <- function(pls_lda, x, ...){

    predctions <- plsgenomics::pls.lda(
        pls_lda$x_train,
        pls_lda$y_train %>% as.numeric() + 1,
        as.data.frame(x),
        pls_lda$ncomp,
        pls_lda$nruncv
    )$predclass

    predictions_df <- as.data.frame(as.numeric(predctions) - 1)
    colnames(predictions_df) <- c("z")
    predictions_df$x <- rep(0,nrow(predictions_df))
    predictions_df$y <- rep(0,nrow(predictions_df))


    return(convert_fg1_int( predictions_df)$z %>% as.factor())
    
}

In [24]:
read_pls_lda_model <- function(pls_lda_obj){
    x_train

}

In [25]:
train_model <- function(
    train_df, 
    train_labels,
    test_df, 
    test_labels,
    ntree = 10,
    outlier_fn = NULL,
    preprocess_fn = NULL,
    weight_fn = targets_to_weights,
    model_id = uuid::UUIDgenerate(),
    ignore_cols = NULL,
    seed = NULL,
    log_string = ""
){
    if(!is.null(seed)){
        set.seed(seed)
    }

    x_train <- train_df %>% as.data.frame()
    x_test <- test_df %>% as.data.frame()
    if(is.function(outlier_fn)){
        x_train <- outlier_fn(x_train)
    }
    if(is.function(preprocess_fn)){
        x_train <- preprocess_fn(x_train)
        x_test <- preprocess_fn(x_test)
    }

    model <- ranger::ranger(
            num.trees = ntree,
            case.weights = weight_fn(train_labels),
            classification = TRUE,
            x=x_train,
            y=train_labels
        )

    if(("Forb" %in% levels(train_labels)) && !("Forb"  %in% levels(test_labels))){
            levels(test_labels) <- c(levels(test_labels), "Forb")
            }

    # create predictions (ranger)
        model_predictions <- predict(
            model, 
            x_test
        )$prediction %>% as.factor()

        # generate the confusion matrix

        confusion_matrix <- caret::confusionMatrix(
            model_predictions, 
            test_labels,
            mode = "everything"
        )

        # generate an id to uniquely identify the model
        #model_id <- uuid::UUIDgenerate()

        # append performance data to the logs for later comparison
        log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = log_string,
            distribition = model_predictions %>% as.factor() %>% table(),
            logpath = "./gs_pls_lda.log")

        # track what levels are associated with the UUID

        # save the model using the model UUID
        save(model, file = paste0("mle/models/gs/", model_id, ".rda"))
        
        return(
            list(
                model = model,
                confusion = confusion_matrix %>% as.list()
                )
        )
}

In [26]:
filter_df_bands <- function(df){
    bands_col_names <- read.csv("assets/band_cols.csv")$names %>% as.character()
    df_cols <- colnames(df) %>% as.character()
    target_cols <- intersect(df_cols, bands_col_names)
    return(df[,target_cols])
}

In [27]:
base_paths <- c(
    "bison_gulch_stratified.csv",
    "img_raw_raw.csv",
    "corrected_and_img_only_bands.csv",
    "grd_raw_raw.csv",
    "img_indices_only.csv",# include veg indices
    "grd_raw_corrected.csv",
    "grd_indices_only.csv"
)
# 

In [28]:
calculate_posterior_weights <- function(validation_path ="figures/merged_validation_s.csv" ){

    validation_df <- read.csv(validation_path, header = TRUE)
    #print(head(validation_df))

    total_observations <- sum(validation_df$validation_counts)
    #print(total_observations)
    weights <- (1/ validation_df$validation_prop)
    #print(validation_df$validation_prop)

    total_by_fg1 <- aggregate(
        x = validation_df$validation_counts,
        by = list(validation_df$key),
        FUN = sum
    )

    fg1_weight_list <- list()

    for( row_idx in seq(nrow(total_by_fg1))){
        name <- total_by_fg1$Group.1[[row_idx]]
        value <- total_by_fg1$x[[row_idx]]
        fg1_weight_list[name] <- value
    }
    
    return(fg1_weight_list)
}

get_posterior_weights_from_targets <- function(target_factor, posterior_weight = calculate_posterior_weights()){
    unbiased_weights <- targets_to_weights(target_factor)

    target_name_char <- target_factor %>% as.character()

    output_weights <- seq_along(target_factor)

    for(i in seq_along(target_factor)){
        if(posterior_weight[[target_name_char[[i]]]] > 0){
            fg1_weight <- 1 / posterior_weight[[target_name_char[[i]]]]
        } else {
            fg1_weight <- 0
        }
        output_weights[[i]] <- unbiased_weights[[i]] * fg1_weight
    }

    return(output_weights)
}

## Hypotheses

Peter
* Increasing number of trees will improve the accuracy/validation mismatch
* bias in 

Ken
* decreasing the number of the trees will decrease the impact of the posterior weighting on chi-squared statistic

In [29]:
outlier_functions <- list(
    clip = load_model("./mle/clip_transform.rda"),
    no_treatment = function(x, ignore_cols = NULL){return(x)}# no transform
)

outlier_treatments <- c(
    "no_treatment"#,
    #"clip"
)

preprocess_functions <- list(
    no_treatment = function(x, ignore_cols = NULL){return(x)},# no transform
    min_max = columnwise_min_max_scale,
    robust = columnwise_robust_scale,
    standard = standardize_df
)

weight_functions <- list(
    posterior = get_posterior_weights_from_targets,
    balanced = targets_to_weights,
    no_treatment = function(x){return(NULL)}# No weights
)
weight_treatments <- c(
    "balanced"#,
    #"no_treatment",
    #"posterior"
)

preprocessing_treatments <- c(
    "standard",
    "min_max",
    "robust",
    "no_treatment"#,
)

In [30]:
num_components <- c(
    1,2,4,6,8,10,12,14,16,18,20,24,32,50#,64,75,100,128,200,256,512,1000
)

In [31]:
test_data <- subset(read.csv("Data/gs/x_test/img_raw_raw.csv"), select = -c(X))
test_labels <- read.csv("Data/gs/y_test/img_raw_raw.csv")$x %>% as.factor()
#train_labels <- read.csv("Data/gs/y_train/img_raw_raw.csv")$x %>% as.factor()

In [32]:
for(filepath in base_paths){
    train_data <- subset(read.csv(paste0("Data/gs/x_train/", filepath)), select = -c(X))
    labels <- read.csv(paste0("Data/gs/y_train/", filepath))$x %>% as.factor()

    for(o_treatment in outlier_treatments){
        for(p_treatment in preprocessing_treatments){
            for(w_treatment in weight_treatments){
                for(n in num_components){


                print(p_treatment)
                print("Number of columns in train and test:")
                print(ncol(train_data))
                print(ncol(test_data))
                print("Number of rows in train and number of labels:")
                print(nrow(train_data))
                print(length(labels))

                model_id <- uuid::UUIDgenerate()
                save_path <- paste0("mle/experiments/gs/", model_id, "/")
                if(!dir.exists(save_path)){
                    dir.create(save_path)
                }

                rf_model_results <- train_model(
                    filter_df_bands(train_data), 
                    labels, 
                    test_data,
                    test_labels,
                    ntree = n,
                    outlier_fn = outlier_functions[[o_treatment]],
                    preprocess_fn = preprocess_functions[[p_treatment]],
                    weight_fn = weight_functions[[w_treatment]],
                    model_id = model_id,
                    seed=61718,
                    log_string = paste(n, filepath, o_treatment, p_treatment, w_treatment)
                )
                print(rf_model_results)

                rf_model <- rf_model_results$model
                acc <- as.list(rf_model_results$confusion$overall)$Accuracy
                print(acc)



                results <- validate_model(
                    rf_model, 
                    save_path, 
                    outlier_processing = outlier_functions[[o_treatment]],
                    transform_type = preprocess_functions[[p_treatment]],
                )

                aggregated_results <- aggregate_results(save_path)

                # calculate validation statistics
                chi2 <- calculate_chi_squared_probability(aggregated_results)
                r2 <- calculate_validation_r2(aggregated_results)

                add_model_to_manifest(
                    model_id = model_id,
                    outlier = o_treatment,
                    preprocessing = p_treatment,
                    source = filepath,
                    weight = w_treatment,
                    n = n,
                    oob_error = rf_model$prediction.error,
                    accuracy = acc,
                    r2 = r2,
                    chi2prob = chi2,
                    logpath="./gs_manifest_pls_only_bands.csv"
                )

                plot_by_pft(
                    aggregated_results,
                    save_path = paste0(save_path, "aggregate.html"),
                    open = FALSE,
                    image_path = NULL
                )
            #
                write_validation_table(
                    aggregated_results,
                    save_path = paste0(save_path, "table.html"),
                    open = FALSE
                )
                }
            }
        }
    }
}

[1] "no_treatment"
[1] "Number of columns in train and test:"
[1] 214
[1] 214
[1] "Number of rows in train and number of labels:"
[1] 108
[1] 108


Warning message in levels(reference) != levels(data):
“longer object length is not a multiple of shorter object length”
Warning message in confusionMatrix.default(model_predictions, test_labels, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”


$model
Ranger result

Call:
 ranger::ranger(num.trees = ntree, case.weights = weight_fn(train_labels),      classification = TRUE, x = x_train, y = train_labels) 

Type:                             Classification 
Number of trees:                  1 
Sample size:                      108 
Number of independent variables:  119 
Mtry:                             10 
Target node size:                 1 
Variable importance mode:         none 
Splitrule:                        gini 
OOB prediction error:             25.00 % 

$confusion
Confusion Matrix and Statistics

                Reference
Prediction       Abiotic Graminoid Lichen Moss ShrubDecid ShrubEvergreen
  Abiotic              8         0      0    0          0              0
  Graminoid            0         0      0    0          0              0
  Lichen               1         0      1    0          0              0
  Moss                 0         0      0    0          0              0
  ShrubDecid           6         0   

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



       x              y          X402.593_5nm       X407.593_5nm      
 Min.   :-149   Min.   :63.81   Min.   :0.002963   Min.   :0.0003612  
 1st Qu.:-149   1st Qu.:63.81   1st Qu.:0.025574   1st Qu.:0.0259253  
 Median :-149   Median :63.81   Median :0.047424   Median :0.0478972  
 Mean   :-149   Mean   :63.81   Mean   :0.044875   Mean   :0.0452952  
 3rd Qu.:-149   3rd Qu.:63.81   3rd Qu.:0.062267   3rd Qu.:0.0628160  
 Max.   :-149   Max.   :63.81   Max.   :0.101671   Max.   :0.1021141  
  X412.593_5nm       X417.593_5nm       X422.593_5nm       X427.593_5nm     
 Min.   :0.003788   Min.   :0.004023   Min.   :0.002517   Min.   :0.004259  
 1st Qu.:0.026049   1st Qu.:0.026182   1st Qu.:0.026527   1st Qu.:0.026813  
 Median :0.047750   Median :0.047706   Median :0.048031   Median :0.048410  
 Mean   :0.045506   Mean   :0.045866   Mean   :0.046254   Mean   :0.046775  
 3rd Qu.:0.063317   3rd Qu.:0.064070   3rd Qu.:0.064752   3rd Qu.:0.065640  
 Max.   :0.112421   Max.   :0.116407   Ma

...

Saving 6.67 x 6.67 in image



Simple feature collection with 1 feature and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.9516 ymin: 63.80698 xmax: -148.9516 ymax: 63.80699
Geodetic CRS:  GCS_unknown
  CLASS_ID   CLASS_NAME AREA                       geometry
9        5 Bisongulch40  674 POLYGON ((-148.9516 63.8069...


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Warning message in chisq.test(validation, prediction):
“Chi-squared approximation may be incorrect”
Warning message in RColorBrewer::brewer.pal(N, "Set2"):
“minimal value for n is 3, returning requested palette with 3 different levels
”
Warning message in RColorBrewer::brewer.pal(N, "Set2"):
“minimal value for n is 3, returning requested palette with 3 different levels
”
Warning message in RColorBrewer::brewer.pal(N, "Set2"):
“minimal value for n is 3, returning requested palette with 3 different levels
”
Warning message in RColorBrewer::brewer.pal(N, "Set2"):
“minimal value for n is 3, returning requested palette with 3 different levels
”
Warning message in RColorBrewer::brewer.pal(N, "Set2"):
“minim

[1] "no_treatment"
[1] "Number of columns in train and test:"
[1] 214
[1] 214
[1] "Number of rows in train and number of labels:"
[1] 108
[1] 108


Warning message in levels(reference) != levels(data):
“longer object length is not a multiple of shorter object length”
Warning message in confusionMatrix.default(model_predictions, test_labels, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”


In [ ]:
sink(NULL)

In [ ]:
print(setdiff(colnames(test_data), colnames(train_data)))